In [1]:
import pandas as pd
import numpy as np

In [78]:
df = pd.DataFrame.from_csv('lending-club-data.csv')

D:\Users\zeyang\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2825: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [79]:
df['safe_loans'] = df['bad_loans'].apply(lambda x: 1 if x == 0 else -1)
del df['bad_loans']

In [80]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [81]:
df = df[features+[target]]

In [82]:
train_idx = pd.read_json('module-5-assignment-2-train-idx.json')[0]
test_idx = pd.read_json('module-5-assignment-2-test-idx.json')[0]

In [83]:
# one-hot encoding
df = pd.get_dummies(df)

In [84]:
train_data = df.iloc[train_idx].reset_index(drop=True)
test_data = df.iloc[test_idx].reset_index(drop=True)

In [85]:
# Implement binary decision tree

In [86]:
def intermediate_node_num_mistakes(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    num_ones = len(labels_in_node[labels_in_node == 1])
    num_zeros = len(labels_in_node[labels_in_node == -1])
    return num_ones if num_zeros >= num_ones else num_zeros

In [87]:
# Test case 1
example_labels = pd.Series([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

# Test case 2
example_labels = pd.Series([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')
    
# Test case 3
example_labels = pd.Series([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


In [88]:
train_data.head(1)

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_n/a
0,-1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [89]:
def best_splitting_feature(data, features, target):
    target_values = data[target]
    best_feature = None
    current_error = 10
    num_data_points = float(len(data))
    
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        right_mistake = intermediate_node_num_mistakes(right_split[target])
        
        error = (left_mistakes + right_mistake) / num_data_points
        if error < current_error:
            current_error = error
            best_feature = feature
    print('Best feature found is {}, the corresponding error is {}'.format(feature, current_error))
    return best_feature

In [90]:
class Node:
    def __init__(self, splitting_features=None, left=None, right=None, is_leaf=False, prediction=None):
        self.splitting_features = splitting_features
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.prediction = prediction

In [91]:
def create_leaf(target_values):
    leaf = Node(is_leaf=True)
    num_ones = len(target_values[target_values == 1])
    num_minus_ones = len(target_values[target_values == -1])
    
    if num_ones >= num_minus_ones:
        leaf.prediction = 1
    else:
        leaf.prediction = -1
    return leaf

In [92]:
def decision_tree_create(data, features, target, current_depth=0, max_depth=10):
    remaining_features = features[:]
    target_values = data[target]
    
    if intermediate_node_num_mistakes(target_values) == 0:
        print('Stopping condition 1 reached.')
        return create_leaf(target_values)
    
    if len(remaining_features) == 0:
        print('Stopping condition 2 reached.')
        return create_leaf(target_values)
    
    if current_depth >= max_depth:
        print('Reached maximum depth. Stopping for now.')
        return create_leaf(target_values)
    
    best_feature = best_splitting_feature(data, remaining_features, target)
    left_split = data[data[best_feature] == 0]
    right_split = data[data[best_feature] == 1]
    remaining_features.remove(best_feature)
    
    print('Split on feature {}, ({}, {})'.format(best_feature, len(left_split), len(right_split)))
    if len(left_split) == len(data):
        print('Creating leaf node')
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print('Creating leaf node')
        return create_leaf(right_split[target])
    
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth+1, max_depth)
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth+1, max_depth)
    
    return Node(splitting_features=best_feature, left=left_tree, right=right_tree, is_leaf=False, prediction=None)

In [93]:
features = df.columns.values.tolist()

In [94]:
features.remove('safe_loans')

In [95]:
my_decision_tree = decision_tree_create(train_data, features, target, 0, 6)

Best feature found is emp_length_n/a, the corresponding error is 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found is emp_length_n/a, the corresponding error is 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found is emp_length_n/a, the corresponding error is 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found is emp_length_n/a, the corresponding error is 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found is emp_length_n/a, the corresponding error is 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found is emp_length_n/a, the corresponding error is 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Reached maximum depth. Stopping for now.
Reached maximum depth. Stopping for now.
Best feature found is emp_length_n/a, the corresponding error is 0.35131195335276966
Split on feature grade_E, (2058, 0)
Creating leaf node
Best feature found is emp_lengt

In [96]:
tree = Node()

In [97]:
def classify(tree, x, annotate=False):
    if tree.is_leaf:
        if annotate:
            print('At leaf, feature {} predicting {}'.format(tree.splitting_features,tree.prediction))
        return tree.prediction
    if annotate:
        print('Split on {} = {}'.format(tree.splitting_features, x[tree.splitting_features]))
    if x[tree.splitting_features] == 1:
        return classify(tree.right, x, annotate)
    return classify(tree.left, x, annotate)

In [98]:
classify(my_decision_tree, test_data.ix[0])

-1

In [99]:
classify(my_decision_tree, test_data.ix[0], True)

Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
At leaf, feature None predicting -1


-1

In [102]:
# Quiz question: What was the feature that my_decision_tree first split on while making the 
# prediction for test_data[0]?

# term_ 36 months

In [103]:
# Quiz question: What was the first feature that lead to a right split of test_data[0]?

# grade_D

In [104]:
# Quiz question: What was the last feature split on before reaching a leaf node 
# for test_data[0]?

# grade_D

In [105]:
def evaluate_classification_error(tree, data, target):
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    return sum(prediction == data[target]) / len(data)

In [106]:
evaluate_classification_error(my_decision_tree, test_data, target)

0.61622145626884961

In [107]:
# Quiz Question: Rounded to 2nd decimal point, what is the classification error of 
# my_decision_tree on the test_data?

# 0.62

In [108]:
def print_stump(tree, name = 'root'):
    split_name = tree.splitting_features # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % tree.prediction)
        return None
    print('                       %s' % name)
    print('         |---------------|----------------|')
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('    ({})                         ({})'.format(
            ('leaf, label: ' + str(tree.left.prediction) if tree.left.is_leaf else 'subtree'),
            ('leaf, label: ' + str(tree.right.prediction) if tree.right.is_leaf else 'subtree')))

In [109]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term_ 36 months == 0]               [term_ 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [110]:
# Quiz Question: What is the feature that is used for the split at the root node?

# term_ 36 months

In [111]:
print_stump(my_decision_tree.left, my_decision_tree.splitting_features)

                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_A == 0]               [grade_A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [112]:
print_stump(my_decision_tree.left.left, my_decision_tree.left.splitting_features)

                       grade_A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_B == 0]               [grade_B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [113]:
# Quiz question: What is the path of the first 3 feature splits considered along the 
# left-most branch of my_decision_tree?

# term_ 36 months -> grade_A -> grade_B

In [56]:
print_stump(my_decision_tree.right, my_decision_tree.splitting_features)

                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_D == 0]               [grade_D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


In [114]:
print_stump(my_decision_tree.right.right, my_decision_tree.right.splitting_features)

(leaf, label: -1)
